**El objetivo de este código es prefiltrar el dataset antes de hacer la búsqueda multimodal** 

In [12]:
import os
print(os.getcwd())

/Users/paulacamprecios/Documents/Master - Pontia/smartshop-advisor/app/data/labelling


In [50]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("bhavikjikadara/e-commerce-products-images")

/Users/paulacamprecios/Documents/Master - Pontia/smartshop-advisor/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 283M/283M [00:10<00:00, 29.4MB/s] 

Extracting files...


In [51]:
print("Path to dataset files:", path)

Path to dataset files: /Users/paulacamprecios/.cache/kagglehub/datasets/bhavikjikadara/e-commerce-products-images/versions/2


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000)  

In [54]:
import pandas as pd

csv_path = "/Users/paulacamprecios/.cache/kagglehub/datasets/bhavikjikadara/e-commerce-products-images/versions/2/styles.csv"

# Leer el CSV
try:
    df = pd.read_csv(csv_path)
except UnicodeDecodeError:
    df = pd.read_csv(csv_path, encoding="ISO-8859-1")

# Revisar primeras filas y columnas
print(df.head())
print(df.columns)

      id gender masterCategory subCategory  articleType baseColour  season  \
0  15970    Men        Apparel     Topwear       Shirts  Navy Blue    Fall   
1  39386    Men        Apparel  Bottomwear        Jeans       Blue  Summer   
2  59263  Women    Accessories     Watches      Watches     Silver  Winter   
3  21379    Men        Apparel  Bottomwear  Track Pants      Black    Fall   
4  53759    Men        Apparel     Topwear      Tshirts       Grey  Summer   

     year   usage                             productDisplayName  
0  2011.0  Casual               Turtle Check Men Navy Blue Shirt  
1  2012.0  Casual             Peter England Men Party Blue Jeans  
2  2016.0  Casual                       Titan Women Silver Watch  
3  2011.0  Casual  Manchester United Men Solid Black Track Pants  
4  2012.0  Casual                          Puma Men Grey T-shirt  
Index(['id', 'gender', 'masterCategory', 'subCategory', 'articleType',
       'baseColour', 'season', 'year', 'usage', 'productDi

**TF-IDF + Random Forest**

In [57]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
df = df.dropna(subset=['productDisplayName', 'articleType']) #dropping nulls (we can do a pipeline)

vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df['productDisplayName'])
y = df['articleType']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = RandomForestClassifier(n_estimators=200, random_state=42)
clf.fit(X_train, y_train)


,n_estimators,200
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [58]:
from sklearn.metrics import classification_report

y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

                        precision    recall  f1-score   support

    Accessory Gift Set       1.00      1.00      1.00        25
            Baby Dolls       0.20      0.50      0.29         2
             Backpacks       0.98      0.96      0.97       161
                Bangle       0.93      1.00      0.97        14
           Basketballs       0.67      1.00      0.80         2
             Bath Robe       0.80      1.00      0.89         4
                 Belts       1.00      1.00      1.00       165
               Blazers       1.00      1.00      1.00         1
           Body Lotion       0.50      1.00      0.67         1
               Booties       1.00      1.00      1.00         3
                Boxers       1.00      0.83      0.91         6
                   Bra       1.00      1.00      1.00        89
              Bracelet       1.00      1.00      1.00        13
                Briefs       1.00      1.00      1.00       159
             Camisoles       1.00      

/Users/paulacamprecios/Documents/Master - Pontia/smartshop-advisor/.venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Users/paulacamprecios/Documents/Master - Pontia/smartshop-advisor/.venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Users/paulacamprecios/Documents/Master - Pontia/smartshop-advisor/.venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted s

In [61]:
# Test phrases
phrases = [
    "I want wide-leg pants",
    "white high boots",
    "blue silk shirt",
    "printed midi skirt"
]

# Transform using the trained vectorizer
X_new = vectorizer.transform(phrases)

# Predict with the trained model
predictions = clf.predict(X_new)

# Display results
for phrase, pred in zip(phrases, predictions):
    print(f"Phrase: '{phrase}' --> Classification: {pred}")

Phrase: 'I want wide-leg pants' --> Classification: Heels
Phrase: 'white high boots' --> Classification: Heels
Phrase: 'blue silk shirt' --> Classification: Tshirts
Phrase: 'printed midi skirt' --> Classification: Skirts


In [64]:
# Test phrases
phrases = [
    "I want wide-leg pants",
    "white high boots",
    "blue silk shirt",
    "printed midi skirt",
    "black leather jacket",
    "red summer dress",
    "striped t-shirt",
    "green hoodie",
    "denim shorts",
    "wool scarf",
    "leather belt",
    "ankle boots",
    "long trench coat",
    "sneakers for running",
    "formal trousers",
    "leather gloves",
    "knit cardigan",
    "polka dot blouse",
    "beige chinos",
    "lace-up boots"
]

# Transform using the trained vectorizer
X_new = vectorizer.transform(phrases)

# Predict with the trained model
predictions = clf.predict(X_new)

# Display results
for phrase, pred in zip(phrases, predictions):
    print(f"Phrase: '{phrase}' --> Classification: {pred}")


Phrase: 'I want wide-leg pants' --> Classification: Heels
Phrase: 'white high boots' --> Classification: Heels
Phrase: 'blue silk shirt' --> Classification: Tshirts
Phrase: 'printed midi skirt' --> Classification: Skirts
Phrase: 'black leather jacket' --> Classification: Jackets
Phrase: 'red summer dress' --> Classification: Dresses
Phrase: 'striped t-shirt' --> Classification: Tshirts
Phrase: 'green hoodie' --> Classification: Heels
Phrase: 'denim shorts' --> Classification: Shorts
Phrase: 'wool scarf' --> Classification: Scarves
Phrase: 'leather belt' --> Classification: Belts
Phrase: 'ankle boots' --> Classification: Heels
Phrase: 'long trench coat' --> Classification: Heels
Phrase: 'sneakers for running' --> Classification: Heels
Phrase: 'formal trousers' --> Classification: Trousers
Phrase: 'leather gloves' --> Classification: Heels
Phrase: 'knit cardigan' --> Classification: Heels
Phrase: 'polka dot blouse' --> Classification: Heels
Phrase: 'beige chinos' --> Classification: Heel

**Word2Vec + Random Forest**

In [67]:
import pandas as pd
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import numpy as np

# Dataset
X_texts = df["productDisplayName"].astype(str).tolist()
y = df["articleType"]

# Tokenizar frases (lista de listas de palabras)
tokenized = [text.lower().split() for text in X_texts]

# Entrenar Word2Vec (o podrías cargar pretrained como GoogleNews, GloVe, FastText)
w2v_model = Word2Vec(sentences=tokenized, vector_size=100, window=5, min_count=2, workers=4)

# Función para obtener embedding promedio de cada frase
def get_sentence_vector(sentence, model):
    words = [w for w in sentence if w in model.wv]
    if len(words) == 0:
        return np.zeros(model.vector_size)
    return np.mean(model.wv[words], axis=0)

X_vectors = np.array([get_sentence_vector(s, w2v_model) for s in tokenized])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_vectors, y, test_size=0.2, random_state=42)

# Clasificador (ejemplo: RandomForest)
clf = RandomForestClassifier(n_estimators=200, random_state=42)
clf.fit(X_train, y_train)

print("Score:", clf.score(X_test, y_test))


ModuleNotFoundError: No module named 'gensim'

**FastText + Random Forest**

In [1]:
!pip install gensim fasttext

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-3.0.1-py3-none-any.whl.metadata (10.0 kB)
Using cached pybind11-3.0.1-py3-none-any.whl (293 kB)
  Created wheel for fasttext-wheel: filename=fasttext_wheel-0.9.2-cp313-cp313-macosx_13_0_universal2.whl size=724630 sha256=93aaf7e58b81777cd3317262b11f26cee13e463dce36f0c095feb05d1a785c98
  Stored in directory: /Users/paulacamprecios/Library/Caches/pip/wheels/d6/41/cd/675080c30f7d666019afefb6494f3fb6530528a9b7ef276102
Successfully built fasttext-wheel
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [fasttext-wheel]


In [2]:
import pandas as pd
import numpy as np
from gensim.models.fasttext import load_facebook_vectors
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# --- Cargar dataset ---
X_texts = df["productDisplayName"].astype(str).tolist()
y = df["articleType"]

# --- Tokenizar ---
tokenized = [text.lower().split() for text in X_texts]

# --- Cargar FastText preentrenado en inglés ---
# Necesitas descargar el modelo primero:
# !wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
fasttext_model = load_facebook_vectors("cc.en.300.bin")  # 1GB aprox

# --- Vectorizar usando promedio de embeddings ---
def get_sentence_vector(sentence, model):
    words = [w for w in sentence if w in model.key_to_index]
    if len(words) == 0:
        return np.zeros(model.vector_size)
    return np.mean(model[words], axis=0)

X_vectors = np.array([get_sentence_vector(s, fasttext_model) for s in tokenized])

# --- Train/Test split ---
X_train, X_test, y_train, y_test = train_test_split(X_vectors, y, test_size=0.2, random_state=42)

# --- Clasificador ---
clf = RandomForestClassifier(n_estimators=200, random_state=42)
clf.fit(X_train, y_train)

print("Accuracy:", clf.score(X_test, y_test))


ModuleNotFoundError: No module named 'gensim'

In [ ]:
# --- Evaluación interactiva ---
def predict_fasttext(sentences, fasttext_model, clf):
    tokenized_sentences = [s.lower().split() for s in sentences]
    vectors = np.array([get_sentence_vector(s, fasttext_model) for s in tokenized_sentences])
    predictions = clf.predict(vectors)
    for s, p in zip(sentences, predictions):
        print(f"🔎 '{s}' → Predicted category: {p}")

# --- Probar con frases ---
test_sentences = [
    "I want white tall boots",
    "Looking for black skinny trousers",
    "Need a red summer dress",
    "Buy running sneakers for gym",
    "Looking for leather jacket"
]

predict_fasttext(test_sentences, fasttext_model, clf)


**DistilBERT / Transformers para clasificación**

In [ ]:
!pip install transformers datasets torch

In [ ]:
from datasets import Dataset
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments
import torch

# --- Preparar dataset ---
dataset = Dataset.from_pandas(df[['productDisplayName','articleType']])
dataset = dataset.rename_column("productDisplayName", "text")
dataset = dataset.rename_column("articleType", "label")

# Mapear labels a números
labels = dataset.unique("label")
label2id = {l:i for i,l in enumerate(labels)}
id2label = {i:l for l,i in label2id.items()}
dataset = dataset.map(lambda x: {"label": label2id[x["label"]]})

# --- Tokenizer ---
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")
def tokenize(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=64)
dataset = dataset.map(tokenize, batched=True)

# --- Split train/test ---
dataset = dataset.train_test_split(test_size=0.2)
train_dataset = dataset["train"]
test_dataset = dataset["test"]

# --- Modelo ---
model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id
)

# --- Entrenamiento ---
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    logging_dir="./logs"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

trainer.train()


In [ ]:
from transformers import pipeline

# --- Pipeline de inferencia ---
inference_pipeline = pipeline(
    "text-classification",
    model=model,
    tokenizer=tokenizer,
    return_all_scores=False
)

def predict_bert(sentences, pipeline, id2label):
    for s in sentences:
        pred = pipeline(s)[0]
        label = pred['label']
        confidence = pred['score']
        print(f"🔎 '{s}' → Predicted category: {id2label[int(label.split('_')[-1])] if label.startswith('LABEL_') else label} ({confidence:.2f})")

# --- Probar con frases ---
test_sentences = [
    "I want white tall boots",
    "Looking for black skinny trousers",
    "Need a red summer dress",
    "Buy running sneakers for gym",
    "Looking for leather jacket"
]

predict_bert(test_sentences, inference_pipeline, id2label)
